In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
sys.path.append("/home/files/feature_orthogonality")
from utils.session_config import setup_gpus

In [2]:
import tensorflow as tf
import cv2
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
setup_gpus(memory_fraction=0.2)

1 Physical GPUs, 1 Logical GPUs


In [4]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, ReLU, BatchNormalization, GlobalAveragePooling2D
from utils.losses import L2_Orthogonal

# Plane mnist classifier

In [ ]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()
train_x = train_x[:,:,:,tf.newaxis]/255.
test_x = test_x[:,:,:,tf.newaxis]/255.
print(train_x.shape)
print(test_x.shape)

In [ ]:
model = tf.keras.Sequential()
model.add(Conv2D(16, (3,3), kernel_regularizer=L2_Orthogonal()))
model.add(ReLU())
model.add(MaxPool2D())
model.add(Conv2D(32, (3,3), kernel_regularizer=L2_Orthogonal()))
model.add(ReLU())
model.add(MaxPool2D())
model.add(Conv2D(32, (3,3), kernel_regularizer=L2_Orthogonal()))
model.add(ReLU())
model.add(GlobalAveragePooling2D())
model.add(Dense(10, activation='softmax', kernel_regularizer='l2'))

'''
model = tf.keras.Sequential()
model.add(Conv2D(32, (3,3), kernel_regularizer=L2_Orthogonal()))
model.add(ReLU())
model.add(MaxPool2D())
model.add(Conv2D(64, (3,3), kernel_regularizer=L2_Orthogonal()))
model.add(ReLU())
model.add(Flatten())
model.add(Dense(10, activation='softmax', kernel_regularizer=L2_Orthogonal()))
'''

In [ ]:
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['acc'])
model.fit(train_x, train_y, epochs=5)
model.evaluate(test_x, test_y)

In [ ]:
model.summary()

In [ ]:
from utils import visualization

visualization.visualize_weight_orthogonality(model)

In [ ]:
model.save("orthogonal_classifier.h5")

# Mnist 3 class (0, 5, 7) classifier

In [5]:
data_0 = np.load("0.npy")
label_0 = np.zeros(len(data_0), np.int32)
data_5 = np.load("5.npy")
label_5 = np.ones(len(data_5), np.int32)
data_7 = np.load("7.npy")
label_7 = np.ones(len(data_7), np.int32)*2

In [6]:
train_x = tf.concat([data_0, data_5, data_7], axis=0)
train_x = tf.cast(train_x, tf.float32)
train_x = train_x[:,:,:,tf.newaxis]/255.
train_y = tf.concat([label_0, label_5, label_7], axis=0)

In [7]:
print(train_x.shape)
print(train_y.shape)

(17609, 28, 28, 1)
(17609,)


In [8]:
x = tf.data.Dataset.from_tensor_slices(train_x)
y = tf.data.Dataset.from_tensor_slices(train_y)
ds = tf.data.Dataset.zip((x, y))
ds = ds.shuffle(len(train_x), reshuffle_each_iteration=False).shuffle(1000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
#(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()
# train_x = train_x[:,:,:,tf.newaxis]/255.
# test_x = test_x[:,:,:,tf.newaxis]/255.
# print(train_x.shape)
# print(test_x.shape)

# Build Model

In [10]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

model = tf.keras.Sequential()
model.add(Conv2D(16, (3,3), kernel_regularizer=L2_Orthogonal()))
model.add(ReLU())
model.add(MaxPool2D())
model.add(Conv2D(16, (3,3), kernel_regularizer=L2_Orthogonal()))
model.add(ReLU())
model.add(MaxPool2D())
model.add(Conv2D(16, (3,3), kernel_regularizer=L2_Orthogonal()))
model.add(ReLU())
model.add(GlobalAveragePooling2D())
model.add(Dense(3, activation='softmax', kernel_regularizer='l2'))

In [11]:
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['acc'])
model.fit(ds, epochs=5)

Epoch 1/5
551/551 [==============================] - 2s 4ms/step - loss: 0.3045 - acc: 0.9259
Epoch 2/5
551/551 [==============================] - 2s 4ms/step - loss: 0.1246 - acc: 0.9880
Epoch 3/5
551/551 [==============================] - 2s 4ms/step - loss: 0.1092 - acc: 0.9916
Epoch 4/5
551/551 [==============================] - 2s 4ms/step - loss: 0.1026 - acc: 0.9918
Epoch 5/5
551/551 [==============================] - 2s 4ms/step - loss: 0.0960 - acc: 0.9930


In [12]:
model.save("mnist-3-cat-classifier.h5")

In [ ]:
from utils import visualization

visualization.visualize_weight_orthogonality(model)

In [ ]:
model.summary()

# Back Etching propagation

In [ ]:
from utils.losses import Inverse_l1_reg

from tensorflow import keras
class Etching_Layer(keras.layers.Layer):
    def __init__(self, lamb):
        super(Etching_Layer, self).__init__()
        self.lamb = lamb

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1]),
            initializer="ones",
            trainable=True,
        )
#     def inverse_l1_reg(self):
#         return -self.lamb * tf.math.reduce_sum(tf.math.abs(self.w))
    def inverse_l1_reg(self):
        return self.lamb * (tf.ones_like(self.w, tf.float32) - self.w)
    def call(self, inputs):
        self.add_loss(self.inverse_l1_reg())
        return inputs*self.w

In [ ]:
layers = []
for layer in model.layers:
    layer.trainable=False
    layers.append(layer)

In [ ]:
print(layers[9].get_weights()[0][:,:1].shape)
print(layers[9].get_weights()[1][:1].shape)

In [ ]:
regressor = Dense(1)
regressor(tf.ones([32,16]))
regressor.set_weights([layers[9].get_weights()[0][:,:1]  , layers[9].get_weights()[1][:1]])
regressor.trainable=False

In [ ]:
input_layer = tf.keras.Input(shape=(28,28,1))
x = layers[0](input_layer) #conv
x = layers[1](x) #relu
x = Etching_Layer(0.1)(x)
x = layers[2](x) #maxpool
x = layers[3](x) #conv
x = layers[4](x) #relu
x = Etching_Layer(0.1)(x)
x = layers[5](x) #maxpool
x = layers[6](x) #conv
x = layers[7](x) #relu
x = Etching_Layer(0.1)(x)
x = layers[8](x) #GAP
x = regressor(x) #regressor

In [ ]:
new_model = tf.keras.Model(input_layer, x)

In [ ]:
new_model.layers[3].get_weights()

In [ ]:
new_model.layers[7].get_weights()

In [ ]:
new_model.layers[11].get_weights()

In [ ]:
new_model.layers[3].trainable=True
new_model.layers[7].trainable=True
new_model.layers[11].trainable=True

In [ ]:
new_model.summary()

In [ ]:
data_0 = np.load("0.npy")
train_x = tf.cast(data_0, tf.float32)
train_x = train_x[:,:,:,tf.newaxis]/255.
ds = tf.data.Dataset.from_tensor_slices(train_x)
ds = ds.shuffle(len(train_x), reshuffle_each_iteration=False).shuffle(1000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for x in ds.take(1):
    print(new_model(x))

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1E-5)
for epoch in range(1):
    for x in ds:
        with tf.GradientTape() as tape:
            pred = new_model(x)
            loss = tf.reduce_sum(pred) + sum(new_model.losses)*10
        gradients = tape.gradient(loss, new_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, new_model.trainable_variables))

In [ ]:
new_model.layers[3].get_weights()

In [ ]:
new_model.layers[7].get_weights()

In [ ]:
new_model.layers[11].get_weights()

# Visualization

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
def get_weight_cosine_similarity(weight):
    w = weight
    if len(w.shape) == 4: #convolution
        H,W,C,N = w.shape
        row_dims = H*W*C
        col_dims = N
    else: #dense
        D,N = w.shape
        row_dims = D
        col_dims = N
    w = tf.reshape(w, (row_dims, col_dims))
    norm = tf.norm(w, axis=0)
    w = w/norm #normalize
    wT = tf.transpose(w)
    correlations = tf.matmul(wT,w)
    return correlations

def get_weight_l2norm(weight):
    w = weight
    if len(w.shape) == 4: #convolution
        H,W,C,N = w.shape
        row_dims = H*W*C
        col_dims = N
    else: #dense
        D,N = w.shape
        row_dims = D
        col_dims = N
    w = tf.reshape(w, (row_dims, col_dims))
    norm = tf.norm(w, axis=0)
    return norm

In [ ]:
weight = model.get_layer("conv2d").get_weights()[0]
correlations = get_weight_cosine_similarity(weight)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 800
sns.heatmap(correlations, vmin=-1, vmax=1, cmap='RdBu_r', center=0, annot=True, fmt='.1f',xticklabels=False, yticklabels=False,annot_kws={"size": 4})

In [ ]:
norm = get_weight_l2norm(weight)
plt.bar(range(correlations.shape[0]), norm)
plt.title('Norm of weights', fontsize=20)
plt.show()

In [ ]:
norm